# Parse Useful Data

In [23]:
'''
Parses CSV files
Inserts values into into PostgreSQL : Docker
'''

import glob
import os
import db_connect
import psycopg2
import pandas as pd

# # Set file path for CSVs
# from set_constants import set_paths
!pwd
csv_directory = '/home/rag/env/exa-data-1/exa-data/data/flattened_csvs'
os.chdir(csv_directory)

/home/rag/env/exa-data-1/exa-data/data/flattened_csvs


In [24]:
def process_csv(csv_file):

    df = pd.read_csv(csv_file)
    num_rows = len(df)

    print(f"this file has {num_rows} rows")

    #print(type(csv_file))

    '''parse the csv file as a nested dictionary and extract the values into variables'''
    dict_from_csv = pd.read_csv(csv_file, header=None, index_col=0).squeeze("columns").to_dict()

    response = dict_from_csv
    entry = response[3]

    for i in range(0,num_rows):

        current_row = entry[i]
        # convert string to dict
        current_row = current_row.strip('\"')
        d = eval(current_row)

        fullUrl = d['fullUrl'] # no nested keys
        resource = d['resource'] # has nested keys
        request = d['request'] # no nested keys

       # shorten the query length
        dq = d.get('resource')
        
        resource_type = (dq.get('resourceType'))
        resource_id = (dq.get('id'))
        
        try:
            meta = (dq.get('meta').get('profile')[0])
        except:
            meta="na"
        
        try:
            identifier_system=(dq.get('identifier'))[0].get('system')
        except:
            identifier_system = "na"
  

        conn = psycopg2.connect(
           database="postgres",
            user='postgres',
            password='postgres',
            host='localhost',
            port= '5432'
        )

        conn.autocommit = True
        cursor = conn.cursor()

        #cursor.execute(sql)
        
        dictionary ={ 'info-1' : (fullUrl, resource_type,
                               'moo_paul@gmail.com', resource_id,
                                meta, 'identifier_use', identifier_system )
        }

        for i in dictionary.values():
            sql2='''insert into patient_info(full_url ,
                  p_resource , request, resource_id,meta, 
                  identifier_use,identifier_system) VALUES{};'''.format(i)

            cursor.execute(sql2)

        conn.commit()
        conn.close()


In [25]:
def main():
    '''Import and parse all CSVs'''
    for csv_file in glob.iglob("*.csv"):
        print(f"processing file {csv_file}")
        print("\n===== New file ========\n")
        process_csv(csv_file)


if __name__ == '__main__':
    '''main driver'''
    main()
    #process_csv('Gus973_Windler79_09e292d4-f186-331c-ed95-c503acabc54e.csv')

processing file Gus973_Windler79_09e292d4-f186-331c-ed95-c503acabc54e.csv

===== New file ========

this file has 330 rows
processing file Della552_Ratke343_b0f40536-9dc8-2ea0-0bbf-467a69f5e3ad.csv

===== New file ========

this file has 556 rows
processing file Lesli455_Champlin946_1d2875d4-f60d-df84-d3e1-12dbafa29bd4.csv

===== New file ========

this file has 1500 rows


KeyboardInterrupt: 